In [2]:
import re
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import datetime

In [3]:
df_train = pd.read_csv('~/Desktop/item_demand_forcast/train.csv')
df_test = pd.read_csv('~/Desktop/item_demand_forcast/test.csv')

In [4]:
df_train.isnull().values.any()

False

In [5]:
df_train.head(10)

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
5,2013-01-06,1,1,12
6,2013-01-07,1,1,10
7,2013-01-08,1,1,9
8,2013-01-09,1,1,12
9,2013-01-10,1,1,9


In [6]:
# 从日期本身抽取出的性质
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['day_of_week'] = df_train['date'].dt.dayofweek + 1
df_train['day_of_month'] = pd.DatetimeIndex(df_train['date']).day
df_train['month'] = pd.DatetimeIndex(df_train['date']).month
df_train['quarter'] = pd.DatetimeIndex(df_train['date']).quarter

In [7]:
df_train.head(10)

,date,store,item,sales,day_of_week,day_of_month,month,quarter
0,2013-01-01,1,1,13,2,1,1,1
1,2013-01-02,1,1,11,3,2,1,1
2,2013-01-03,1,1,14,4,3,1,1
3,2013-01-04,1,1,13,5,4,1,1
4,2013-01-05,1,1,10,6,5,1,1
5,2013-01-06,1,1,12,7,6,1,1
6,2013-01-07,1,1,10,1,7,1,1
7,2013-01-08,1,1,9,2,8,1,1
8,2013-01-09,1,1,12,3,9,1,1
9,2013-01-10,1,1,9,4,10,1,1


In [8]:
df_train.dtypes

date            datetime64[ns]
store                    int64
item                     int64
sales                    int64
day_of_week              int64
day_of_month             int64
month                    int64
quarter                  int64
dtype: object

In [68]:
# 统计量
#pre_process
df_train_item_store_sorted = df_train.sort_values(by=['item', 'store', 'date'])
df_train_item_sorted = df_train[['item', 'date','sales']].groupby(['item', 'date']).sum().sort_values(by=['item', 'date']).reset_index()
df_train_store_sorted = df_train[['store', 'date','sales']].groupby(['store', 'date']).sum().sort_values(by=['store', 'date']).reset_index()

#item past week
temp_col = [i for i in range(7)]
for i in range(7):
    df_train_item_sorted[temp_col[i]] = df_train_item_sorted.loc[
        (df_train_item_sorted['item']==df_train_item_sorted['item'].shift(-i)) &
        (df_train_item_sorted['date']==df_train_item_sorted['date'].shift(-i) - datetime.timedelta(days=i))
        , 'sales']
    df_train_item_sorted[temp_col[i]] = df_train_item_sorted[temp_col[i]].shift(i)
df_train_item_sorted['item_past_week_ave'] = df_train_item_sorted[temp_col].mean(axis=1)
df_train_item_sorted['item_past_week_max'] = df_train_item_sorted[temp_col].max(axis=1)
df_train_item_sorted['item_past_week_min'] = df_train_item_sorted[temp_col].min(axis=1)
df_train_item_sorted['item_past_week_std'] = df_train_item_sorted[temp_col].std(axis=1)
df_train_item_sorted.drop(columns=temp_col, inplace=True)

#item past montn (30 days)
temp_col = [i for i in range(30)]
for i in range(30):
    df_train_item_sorted[temp_col[i]] = df_train_item_sorted.loc[
        (df_train_item_sorted['item']==df_train_item_sorted['item'].shift(-i)) &
        (df_train_item_sorted['date']==df_train_item_sorted['date'].shift(-i) - datetime.timedelta(days=i))
        , 'sales']
    df_train_item_sorted[temp_col[i]] = df_train_item_sorted[temp_col[i]].shift(i)
df_train_item_sorted['item_past_month_ave'] = df_train_item_sorted[temp_col].mean(axis=1)
df_train_item_sorted['item_past_month_max'] = df_train_item_sorted[temp_col].max(axis=1)
df_train_item_sorted['item_past_month_min'] = df_train_item_sorted[temp_col].min(axis=1)
df_train_item_sorted['item_past_month_std'] = df_train_item_sorted[temp_col].std(axis=1)
df_train_item_sorted.drop(columns=temp_col, inplace=True)

#join to df_train
df_train = df_train.merge(df_train_item_sorted.drop(columns='sales'), 
                    left_on=['item', 'date'], right_on=['item', 'date'], how='outer')

In [70]:
df_train.head(100)

,date,store,item,sales,day_of_week,day_of_month,month,quarter,item_past_week_ave,item_past_week_max,item_past_week_min,item_past_week_std,item_past_month_ave,item_past_month_max,item_past_month_min,item_past_month_std
0,2013-01-01,1,1,13,2,1,1,1,133.000000,133.0,133.0,NaN,133.000000,133.0,133.0,NaN
1,2013-01-01,2,1,12,2,1,1,1,133.000000,133.0,133.0,NaN,133.000000,133.0,133.0,NaN
2,2013-01-01,3,1,19,2,1,1,1,133.000000,133.0,133.0,NaN,133.000000,133.0,133.0,NaN
3,2013-01-01,4,1,10,2,1,1,1,133.000000,133.0,133.0,NaN,133.000000,133.0,133.0,NaN
4,2013-01-01,5,1,11,2,1,1,1,133.000000,133.0,133.0,NaN,133.000000,133.0,133.0,NaN
5,2013-01-01,6,1,20,2,1,1,1,133.000000,133.0,133.0,NaN,133.000000,133.0,133.0,NaN
6,2013-01-01,7,1,7,2,1,1,1,133.000000,133.0,133.0,NaN,133.000000,133.0,133.0,NaN
7,2013-01-01,8,1,16,2,1,1,1,133.000000,133.0,133.0,NaN,133.000000,133.0,133.0,NaN
8,2013-01-01,9,1,11,2,1,1,1,133.000000,133.0,133.0,NaN,133.000000,133.0,133.0,NaN
9,2013-01-01,10,1,14,2,1,1,1,133.000000,133.0,133.0,NaN,133.000000,133.0,133.0,NaN


In [ ]:
# to_do

#store past week

#store past month

#store_item past week

#store_item past month